![image](https://www.kalw.org/sites/kalw/files/styles/medium/public/201712/AAEAAQAAAAAAAAf_AAAAJDNjODQwOGE0LTRlM2YtNGJjOS1hNmQxLWNlODFmNjEwNDU3OQ-1.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
df.head().style.background_gradient(cmap='RdGy')

In [ ]:

rslt_df = df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)]
rslt_df2 = df[(df['toxic'] == 1) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)]
new1 = rslt_df[['id', 'comment_text', 'toxic']].iloc[:23891].copy() 
new2 = rslt_df2[['id', 'comment_text', 'toxic']].iloc[:946].copy()
new = pd.concat([new1, new2], ignore_index=True)

In [ ]:
lens = new.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

In [ ]:
lens.hist();

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new["comment_text"], new['toxic'], test_size=0.33)

# Feature hashing 

also known as the hashing trick (by analogy to the kernel trick), is a fast and space-efficient way of vectorizing features

This one is designed to be as memory efficient as possible.
Instead of storing the tokens as strings, the vectorizer applies the hashing trick to encode them as numerical indexes.

* {n-gram → hash(n-gram)%2^ㄻ}
* Has collisions but works in practice
* sklearn.feature_extraction.text.HashingVectorizer
* Implement in vowpal wabbit library.

Link to my work(Implementation using Vowpal Wabbit library) - https://github.com/Bavalpreet/NLP/blob/master/nlp_vw.ipynb

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer(n_features=2**23)
X1 = vectorizer.fit_transform(X_train)
X_test1= vectorizer.transform(X_test)

# SVMSMOTE

SVM-SMOTE, an **over-sampling** technique, is used to investigate how well it handles the trade-off. 
SMOTE, its ancestor, is a popular over-sampling technique which balances class distribution by synthetically generating new minority class instances along directions from existing minority class instances towards their nearest neighbours.
SVM-SMOTE focuses on generating new minority class instances near borderlines with SVM so as to help establish boundary between classes.

![svmsmote](https://miro.medium.com/fit/c/1838/551/1*5t_gGECrUDNV2NiZIovZBw.png)

In [ ]:
print('Original dataset shape %s' % Counter(y_train))
sm = SVMSMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X1, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


![svmsmote](https://miro.medium.com/max/2124/1*wLy-jPchisCrYWKOv_jK_g.png)

LOGISTIC REGRESSION

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
clf2 = LogisticRegression(C=0.1, solver='sag')
scores = cross_val_score(clf2, X_res,y_res, cv=5,scoring='f1_weighted')
scores

In [ ]:
y_p1 = clf2.fit(X_res, y_res).predict(X_test1)
from sklearn.metrics import accuracy_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_p1)
print('Accuracy: %f' % accuracy)

In [ ]:
# finding confidence interval
import numpy as np
z=1.96
interval = z * np.sqrt( (0.908137 * (1 - 0.908137)) / y_test.shape[0])
interval

SVC

In [ ]:
from sklearn.svm import SVC
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf,X_res,y_res, cv=5)
scores

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

y_p2 = clf.fit(X_res, y_res).predict(X_test1)

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_p2)
print('Accuracy: %f' % accuracy)

import numpy as np
# finding confidence interval
z=1.96
interval = z * np.sqrt( (0.963279 * (1 - 0.963279)) / y_test.shape[0])
interval